In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import joblib
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import string

In [2]:
True_df = pd.read_csv(r"C:\Users\shabb\Desktop\Fake_News_detector\data\True.csv")
Fake_df = pd.read_csv(r"C:\Users\shabb\Desktop\Fake_News_detector\data\Fake.csv")

In [3]:
True_df.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [4]:
Fake_df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [5]:
True_df.isna().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [6]:
Fake_df.isna().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [7]:
True_df["Label"] = 1
Fake_df["Label"]=0

In [8]:
True_df.head()

,title,text,subject,date,Label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [9]:
Fake_df.head()

,title,text,subject,date,Label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [10]:
train_df = pd.concat([True_df,Fake_df]).sample(frac=1).reset_index(drop=True)
train_df

,title,text,subject,date,Label
0,Israeli Minister Calls For ‘Civil Targeted Ki...,"In Israel, dehumanizing Palestinians and calli...",News,"March 30, 2016",0
1,Why The NRA Could Be In Big Trouble For Break...,The National Rifle Association may be in viola...,News,"January 3, 2016",0
2,"Thanks To Capitalism, It Just Got A Lot Harde...",Just like with every presidential inauguration...,News,"December 12, 2016",0
3,Trump to sign Russia sanctions bill: White House,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"July 29, 2017",1
4,California Governor Brown pushes big water pro...,"SACRAMENTO, Calif. (Reuters) - California Gove...",politicsNews,"January 14, 2016",1
...,...,...,...,...,...
44893,BUSTED! FOX NEWS REPORTER Caught Twisting Poll...,Is anyone else sick to death of the bias on FO...,politics,"Sep 2, 2016",0
44894,OBAMA RELEASES “CRAFTY” STATEMENT On Wiretappi...,Can you pick this statement apart and let us k...,politics,"Mar 4, 2017",0
44895,LOL! DONALD TRUMP JR. Has HILARIOUS Response T...,"In her new book Hacks , Donna Brazile express...",left-news,"Nov 8, 2017",0
44896,Merkel offers to work with Trump on basis of d...,BERLIN (Reuters) - German Chancellor Angela Me...,politicsNews,"November 9, 2016",1


In [11]:
train_df['Label'].value_counts()

Label
0    23481
1    21417
Name: count, dtype: int64

In [15]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"https\S+","",text)
    text = re.sub(r"d\+","",text)
    text = text.translate(str.maketrans('', '', string.punctuation))  
    text = re.sub(r"\s+", " ", text)  
    return text.strip()

In [16]:
train_df['cleaned_text'] = train_df['text'].apply(clean_text)

In [17]:
train_df

,title,text,subject,date,Label,cleaned_text
0,Israeli Minister Calls For ‘Civil Targeted Ki...,"In Israel, dehumanizing Palestinians and calli...",News,"March 30, 2016",0,in israel dehumanizing palestinians and callin...
1,Why The NRA Could Be In Big Trouble For Break...,The National Rifle Association may be in viola...,News,"January 3, 2016",0,the national rifle association may be in viola...
2,"Thanks To Capitalism, It Just Got A Lot Harde...",Just like with every presidential inauguration...,News,"December 12, 2016",0,just like with every presidential inauguration...
3,Trump to sign Russia sanctions bill: White House,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"July 29, 2017",1,washington reuters us president donald trump i...
4,California Governor Brown pushes big water pro...,"SACRAMENTO, Calif. (Reuters) - California Gove...",politicsNews,"January 14, 2016",1,sacramento calif reuters california governor j...
...,...,...,...,...,...,...
44893,BUSTED! FOX NEWS REPORTER Caught Twisting Poll...,Is anyone else sick to death of the bias on FO...,politics,"Sep 2, 2016",0,is anyone else sick to death of the bias on fo...
44894,OBAMA RELEASES “CRAFTY” STATEMENT On Wiretappi...,Can you pick this statement apart and let us k...,politics,"Mar 4, 2017",0,can you pick this statement apart and let us k...
44895,LOL! DONALD TRUMP JR. Has HILARIOUS Response T...,"In her new book Hacks , Donna Brazile express...",left-news,"Nov 8, 2017",0,in her new book hacks donna brazile expressed ...
44896,Merkel offers to work with Trump on basis of d...,BERLIN (Reuters) - German Chancellor Angela Me...,politicsNews,"November 9, 2016",1,berlin reuters german chancellor angela merkel...


In [18]:
X = train_df['cleaned_text']
y = train_df['Label']

In [19]:
X_train,X_test,Y_train,Y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [20]:
Y_train.value_counts()

Label
0    18774
1    17144
Name: count, dtype: int64

In [21]:
model = Pipeline([
        ('tfidf', TfidfVectorizer(stop_words='english', max_df=0.8)),
        ('model', RandomForestClassifier(n_estimators=100,random_state=42))
    ])

In [22]:
model.fit(X_train,Y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer(max_df=0.8, stop_words='english')),
                ('model', RandomForestClassifier(random_state=42))])

In [34]:
model.score(X_test,Y_test)

0.9874164810690423

In [27]:
y_pred = model.predict(X_test)

In [29]:
print(classification_report(Y_test,y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4707
           1       0.99      0.99      0.99      4273

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



In [25]:
predictions = model.predict(X_test)

In [26]:
for i, pred in enumerate(predictions[:10]):
    label = "Fake" if pred == 0 else "Real"
    print(f"News Text:\n{X_test.iloc[i]}\nPrediction: {label}\n{'-'*60}")


News Text:
the former fox news contributor and editor of the weekly standard bill kristol took to twitter to appease his newfound liberal friends with a disgusting tweet mocking devout christian and genuine conservative vice president mike pence ever since president trump won the election kristol and his nevertrump friends have become unhinged with hate for the president and for anyone he surrounds himself with yesterday vice president mike pence made a surprise visit to the troops in wartorn afghanistan to roll out president trump s new fight to win policypence addressed the troops at a bagram airfield hangar telling them that before he made his trip he asked president trump what message he wanted to be delivered to the troops here is pence s touching message to the troops from president trumppence told the troops before i left the white house yesterday i asked trump if he had a message for our troops here in afghanistan and he looked at me and said tell them i love them and he didwat

array([1], dtype=int64)

In [33]:
joblib.dump(model,r"C:\Users\shabb\Desktop\Fake_News_detector\model\Fake_news_model.pkl")

['C:\\Users\\shabb\\Desktop\\Fake_News_detector\\model\\Fake_news_model.pkl']